채용포털사이트 사람인 크롤링 하기

In [6]:
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests
import re

#검색키워드 분류
search_n = input("검색키워드를 입력하세요>")


#지역코드는 서울(101000) default선언!
#city_n01 = input("검색할 첫번째 도시를 입력하세요")
#city_n02 = input("검색할 두번째 도시를 입력하세요")
#city_n03 = input("검색할 세번째 도시를 입력하세요")



#DF선언
saramIn_df = pd.DataFrame()
saramIn_df_temp = pd.DataFrame()
saramIn_df_temp = saramIn_df_temp.append(
                    {"coID":"",#c
                     "coIndx":"",#c
                     "coNm":"",#c
                     "city":"",#c
                     "cond_a":"",#c
                     "cond_b":"",#c
                     "cond_c":"",#c
                     "title":"", #c
                     "region":"",#c
                     "r_date":""
                        
                    },ignore_index=True

)

#URL 가져오기!!
driver = webdriver.Chrome("C:\\dev\\chromedriver.exe")

driver.get("http://www.saramin.co.kr/zf_user/search/recruit?searchType=search&searchword="+search_n+"&loc_mcd=101000&company_cd=&panel_type=&search_optional_item=y&search_done=y&panel_count=y&recruitPage=&recruitSort=relation&recruitPageCount=40&inner_com_type=&quick_apply=")
html = driver.page_source
soup = bs(html, "html.parser")


#불러올 데이터 총 개수 찾기!
cnt_result = soup.find("span", class_="cnt_result").text
cnt_result = re.sub(r"총 |건","",cnt_result)
cnt_result = cnt_result.split(",")
totCnt =""
for i in range(len(cnt_result)):
    totCnt += cnt_result[i]
totCnt = int(totCnt)
#불러올 데이터 수
totCnt
#불러올 페이지수
totPage = totCnt//50+1
#print(totPage)

#페이지 돌리기(totPage)
for page in range(1,totPage+1):
    driver.get("http://www.saramin.co.kr/zf_user/search/recruit?searchType=search&searchword="+search_n+"&loc_mcd=101000&company_cd=&panel_type=&search_optional_item=y&search_done=y&panel_count=y&recruitPage=&recruitSort=relation&recruitPageCount=40&inner_com_type=&quick_apply=")
    p_html = driver.page_source
    p_soup = bs(html, "html.parser")
    
    comp_list =p_soup.find_all("div", class_="item_recruit")
    
    for view in range(len(comp_list)):#len(comp_list)
        #column 값들 가져오기!
        coID = comp_list[view].find("div",class_="area_corp_info").find("a").get("href")
        coID = re.findall(r"[0-9]{10}",coID)[0]
        coIndx = comp_list[view].find("h2",class_="job_tit").find("a").get("href")
        coIndx = re.findall(r"[0-9]{8}",coIndx)[0]
        coNm = comp_list[view].find("div", class_="area_corp").strong.a.get("title")
        city = comp_list[view].find("div", class_="job_condition").find_all("span")[0].find_all("a")[0].text
        if len(comp_list[view].find("div", class_="job_condition").find_all("span")[0].find_all("a")) == 2:
            region = comp_list[view].find("div", class_="job_condition").find_all("span")[0].find_all("a")[1].text
        else:
            region =""
        
        if len(comp_list[view].find("div", class_="job_condition").find_all("span"))>= 4:
            cond_a = comp_list[view].find("div", class_="job_condition").find_all("span")[1].text
            cond_a = re.sub(r"↑","",cond_a)
            cond_a = re.sub(r"·","",cond_a)
            cond_a = re.sub(r" ","",cond_a)
            cond_b = comp_list[view].find("div", class_="job_condition").find_all("span")[2].text
            cond_b = re.sub(r"↑","",cond_b)
            cond_c = comp_list[view].find("div", class_="job_condition").find_all("span")[3].text
            cond_c = re.sub(r"↑","",cond_c)
            cond_c = re.sub(r"·","",cond_c)
            cond_c = re.sub(r" ","",cond_c)
            
            
            
            #cond_c

        elif len(comp_list[view].find("div", class_="job_condition").find_all("span"))< 4:
            cond_a =  comp_list[view].find("div", class_="job_condition").find_all("span")[1].text
            cond_a = re.sub(r"↑","",cond_a)
            cond_a = re.sub(r"·","",cond_a)
            cond_a = re.sub(r" ","",cond_a)
            cond_b = comp_list[view].find("div", class_="job_condition").find_all("span")[2].text
            cond_b = re.sub(r"↑","",cond_b)
            cond_c = ""
        
        title = comp_list[view].find("h2").find("a").get("title")
        title = re.sub(r" ","",title)
        r_date = comp_list[view].find("span",class_="date").text
        
        #print(coIndx)
        #드라이버종료
        #정보추가얻기 #드라이버 재시작
        
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("coID")] =coID
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("coIndx")] =coIndx
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("coNm")] =coNm
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("city")] =city
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("cond_a")] =cond_a
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("cond_b")] =cond_b
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("cond_c")] =cond_c
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("title")] =title
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("region")] =region
        saramIn_df_temp.iat[0,saramIn_df_temp.columns.get_loc("r_date")] = r_date
        saramIn_df = saramIn_df.append(saramIn_df_temp,ignore_index=True)
        
        saramIn_df_temp.drop(0)
        
        
        
    

saramIn_df

#파일로 출력하기
saramIn_df.to_csv("saramIn_df_"+search_n+".csv")





검색키워드를 입력하세요>pco


In [7]:
saramIn_df.to_csv("saramIn_df_"+search_n+".csv")

In [8]:
df = pd.read_csv("saramIn_df_pco.csv")
df

,Unnamed: 0,city,coID,coIndx,coNm,cond_a,cond_b,cond_c,r_date,region,title
0,0,서울,2148213380,38378383,(사)한국PCO협회,경력2년,초대졸,정규직,~ 07/19(일),강남구,(사)한국PCO협회사무국직원채용
1,1,서울,1978800641,38196624,주식회사 티알큐,경력3~10년,대졸,정규직,~ 07/21(화),노원구,전시.이벤트.세미나기획분야PCO경력사원채용
2,2,서울,2068687079,38274136,(주)더홀릭스,경력,초대졸,정규직,~ 08/07(금),마포구,㈜더홀릭스경력직PCO정규직채용
3,3,서울,1058733199,38334612,(주)피스코퍼스,신입경력,초대졸,정규직,~ 08/18(화),마포구,"[피스코퍼스]컨벤션,국제회의,심포지엄기획"
4,4,서울,1058754627,38446195,주식회사 위드바톤,경력,학력무관,프리랜서,~ 07/31(금),강서구,국제회의업무서브계약직모집
5,5,서울,1018197530,38388394,(주)파워피티,신입경력,초대졸,정규직,채용시,마포구,컨벤션·세미나·데모데이·행사운영정규직채용공고
6,6,서울,1058774604,38229598,(주)스완커뮤니케이션,경력1년,학력무관,정규직,~ 07/28(화),마포구,MICE/Creative한기획자정규직모집공고
7,7,서울,2048626455,38328176,(주)엠앤씨커뮤니케이션즈,신입,학력무관,정규직,~ 07/17(금),강남구,㈜엠앤씨2020년도신입채용공채(MICE/컨벤션)
8,8,서울,1298694026,38266347,(주)리디아이앤엠,경력2년,고졸,정규직계약직,~ 08/04(화),강남구,삼성역컨벤션기획사/분야별모집/워라벨
